# Smart Auto Assist

**Documentation or Blog:** -- https://medium.com/@argowtham3030/smart-auto-assist-55b2f5cb66f1

**Video Explanation** -- https://youtu.be/TfZaIMW6x6I

**WorkFlow**

![](https://raw.githubusercontent.com/argowthamkumar/kaggle/refs/heads/main/capstone/images/flowchart.jpg)

**Install Required Libraries**

In [1]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0' 

# Install necessary libraries for Image Processing, RAG, Grounding, Agent
!pip install -q -U langchain langchain-core langchain-community langchain-google-genai langgraph google-generativeai chromadb pypdf python-dotenv Pillow requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.6

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

# Image Analysis code

In [4]:
# Import libraries
import requests
import PIL
#from PIL import Image
import io
import google.generativeai as genai

# Define function to load image from URL
def load_image_from_url(image_url):
    response = requests.get(image_url, stream=True)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    image = PIL.Image.open(io.BytesIO(response.content)).convert("RGB")
    return image

# Configure the Gemini API
#GOOGLE_API_KEY = "YOUR_API_KEY" # Replace with your actual API key
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')

# Update the process_image function
def process_image(image_url):
    """
    Processes an image URL, describes its contents using the Gemini model, and returns the description.
    """
    try:
        # Load the image
        image = load_image_from_url(image_url)

        # Prepare the prompt for the Gemini model
        prompt = "Describe the contents of this image and if the image contains warning lights, indicate the Severity of warning in bold"

        # Generate content using the Gemini model
        response = model.generate_content([prompt, image])

        # Return the description from the Gemini model's response
        return response.text
    except Exception as e:
        return f"Error processing image: {e}"


In [5]:
#process_image("https://raw.githubusercontent.com/argowthamkumar/kaggle/refs/heads/main/capstone/images/low_fuel.jpg")

## Retrieval augmented generation (RAG)

* Load PDF from URL

In [6]:
import os
import google.generativeai as genai
from google.colab import userdata

import requests
import tempfile

# Configure the Gemini API
genai.configure(api_key=GOOGLE_API_KEY)

EMBEDDING_MODEL = "models/embedding-001"
GENERATION_MODEL = "gemini-1.5-flash" 

# --- Configuration ---
# We need the URL to the RAW PDF file content
raw_pdf_url = "https://raw.githubusercontent.com/argowthamkumar/kaggle/main/capstone/manual.pdf" 

print(f"Attempting to download PDF from: {raw_pdf_url}")

# --- Download the PDF ---
downloaded_pdf_path = None # Initialize path variable
temp_pdf_file = None      # Initialize temp file object variable

try:
    response = requests.get(raw_pdf_url, stream=True)
    response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

    # Create a temporary file to store the PDF content
    # delete=False is important because PyPDFLoader needs to open the file by path later
    # suffix=".pdf" helps identify the file type, though not strictly necessary for PyPDFLoader
    temp_pdf_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    with temp_pdf_file as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
        downloaded_pdf_path = temp_pdf_file.name # Get the path to the temporary file
    print(f"PDF successfully downloaded and saved to temporary file: {downloaded_pdf_path}")

except requests.exceptions.RequestException as e:
    print(f"Error downloading PDF: {e}")
    # Clean up the temporary file if it was created but download failed mid-way
    if downloaded_pdf_path and os.path.exists(downloaded_pdf_path):
        os.remove(downloaded_pdf_path)
    raise # Re-raise the exception to stop execution

except Exception as e:
    # Catch any other potential errors during file handling
    print(f"An error occurred: {e}")
    if downloaded_pdf_path and os.path.exists(downloaded_pdf_path):
        os.remove(downloaded_pdf_path)
    raise

# --- Check if download path is set ---
if not downloaded_pdf_path:
     raise FileNotFoundError("PDF download failed or temporary file path was not set.")


Attempting to download PDF from: https://raw.githubusercontent.com/argowthamkumar/kaggle/main/capstone/manual.pdf
PDF successfully downloaded and saved to temporary file: /tmp/tmpbj_d4atu.pdf


**Text Splitter**

In [7]:

# Load and Process the PDF
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Ensure the downloaded_pdf_path exists from the previous cell
if 'downloaded_pdf_path' not in locals() or not downloaded_pdf_path or not os.path.exists(downloaded_pdf_path):
     raise NameError("Variable 'downloaded_pdf_path' is not defined or the file doesn't exist. "
                     "Make sure Cell 3 ran successfully.")

print(f"Loading PDF from temporary file: {downloaded_pdf_path}")
loader = PyPDFLoader(downloaded_pdf_path) # Use the temporary file path

pages = []
try:
    # Load the PDF pages into LangChain Documents
    pages = loader.load()
    print(f"Loaded {len(pages)} pages from the PDF.")
finally:
    # --- Clean up the temporary file ---
    # It's crucial to delete the temp file after PyPDFLoader is done with it.
    print(f"Cleaning up temporary file: {downloaded_pdf_path}")
    if os.path.exists(downloaded_pdf_path):
        os.remove(downloaded_pdf_path)
        print("Temporary file removed.")
    else:
        print("Temporary file not found (already removed or error occurred).")
    # Optional: Clear the variable to prevent accidental reuse
    # del downloaded_pdf_path


# --- Split documents into smaller chunks ---
if not pages:
     print("Warning: No pages were loaded from the PDF. Splitting step will be skipped.")
     chunks = []
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Maximum size of each chunk (in characters)
        chunk_overlap=100, # Number of characters to overlap between chunks
        length_function=len,
        add_start_index=True, # Adds the start index of the chunk in the original document
    )

    # Split the documents loaded from the PDF
    chunks = text_splitter.split_documents(pages)
    print(f"Split the document into {len(chunks)} chunks.")
   

Loading PDF from temporary file: /tmp/tmpbj_d4atu.pdf
Loaded 408 pages from the PDF.
Cleaning up temporary file: /tmp/tmpbj_d4atu.pdf
Temporary file removed.
Split the document into 1036 chunks.


# Embedding

In [8]:
#Initialize Embeddings Model
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Create an instance of the Gemini Embedding model
embeddings = GoogleGenerativeAIEmbeddings(
    model=EMBEDDING_MODEL,
    task_type="retrieval_document",
    google_api_key=GOOGLE_API_KEY 
)
# task_type can be: retrieval_query, retrieval_document, semantic_similarity, classification, clustering

print("Gemini Embeddings model initialized.")


Gemini Embeddings model initialized.


# Vector Store  Chroma DB

In [9]:

# Setup ChromaDB Vector Store
from langchain_community.vectorstores import Chroma

# --- ChromaDB Configuration ---
CHROMA_PATH = "chroma_db_pdf" # Directory to store ChromaDB data
COLLECTION_NAME = "pdf_knowledge_base" # Name of the collection within ChromaDB

print(f"Setting up ChromaDB persistence directory: {CHROMA_PATH}")
print(f"Using collection name: {COLLECTION_NAME}")

# --- Create or Load the Vector Store ---
# This will create the directory if it doesn't exist and load the data if it does.
# It embeds the chunks and stores them in ChromaDB.
# This step can take some time depending on the number of chunks and API limits.
print("Creating/loading ChromaDB vector store and embedding chunks...")
try:
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=CHROMA_PATH,
        collection_name=COLLECTION_NAME
    )
    print("Vector store created/loaded successfully.")
    # Explicitly persist changes (good practice, though often handled by from_documents)
    vectorstore.persist()
    print("Vector store persisted.")
    # Check the number of items in the store
    print(f"Number of vectors in store: {vectorstore._collection.count()}")

except Exception as e:
    print(f"\nError during vector store creation/embedding: {e}")
    print("Potential issues:")
    print("- API key limits reached (check your quota in Google Cloud Console).")
    print("- Network connectivity issues.")
    print("- Invalid API key or API not enabled.")
    # Depending on the error, you might need to retry or troubleshoot API access.
    # For rate limit errors, you might need to add delays between embedding calls (more complex setup).
    raise # Re-raise the exception to stop execution if embedding fails critically


Setting up ChromaDB persistence directory: chroma_db_pdf
Using collection name: pdf_knowledge_base
Creating/loading ChromaDB vector store and embedding chunks...
Vector store created/loaded successfully.
Vector store persisted.
Number of vectors in store: 1036


/tmp/ipykernel_31/723472596.py:25: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


**Model for RAG**

In [10]:

# Initialize the Gemini LLM for Generation
from langchain_google_genai import ChatGoogleGenerativeAI

# Create an instance of the Gemini LLM
llm = ChatGoogleGenerativeAI(
    model=GENERATION_MODEL,
    temperature=0.3, # Lower temperature for more factual answers
    #convert_system_message_to_human=True, # Helps manage prompts for some models
    google_api_key=GOOGLE_API_KEY 
)

print("Gemini LLM for generation initialized.")

Gemini LLM for generation initialized.


# Retrieval augmented generation (RAG) 

In [11]:

# Build the RAG Chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- Create a Retriever ---
# The retriever fetches relevant documents from the vector store based on the query.
my_retriever = vectorstore.as_retriever(
    search_type="similarity", # Other options: "mmr", "similarity_score_threshold"
    search_kwargs={'k': 5} # Retrieve top 5 most relevant chunks
)
print("Retriever created from vector store.")

# --- Define the Prompt Template ---
# This template structures the input to the LLM, providing context and the question.
RAG_PROMPT_TEMPLATE = """
CONTEXT:
{context}

QUESTION:
{question}

Based only on the context provided, answer the question. If the context doesn't contain the answer, state that you cannot answer based on the provided information.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)
print("RAG prompt template created.")


# --- Create the RAG Chain using LangChain Expression Language (LCEL) ---

# This function formats the retrieved documents into a single string.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG pipeline
rag_chain = (
    # RunnablePassthrough assigns the original question to the 'question' key
    # retriever | format_docs assigns the formatted context to the 'context' key
    {"context": my_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt  # Pass the dictionary to the prompt template
    | llm         # Pass the formatted prompt to the LLM
    | StrOutputParser() # Parse the LLM output into a string
)

print("RAG chain created successfully.")

Retriever created from vector store.
RAG prompt template created.
RAG chain created successfully.


In [12]:
# --- Function to ask a question ---
def car_manual_rag(query):
    print(f"\n🤔 Query: {query}")
    try:
        # Invoke the RAG chain
        answer = rag_chain.invoke(query)
        print("\n✅ Answer:")
        print(answer)
        return answer

    except Exception as e:
        print(f"\n❌ Error processing query: {e}")

#car_manual_rag("how to set seat heating?")

# Grounding with Google

In [13]:
from IPython.display import display, Image, Markdown

config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

def show_response(response):
    for p in response.content.parts:
        if p.text:
            return p.text
        elif p.inline_data:
            return p.inline_data
        else:
            return p.to_json_dict()

def google_ground_search(query):
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=query,
        config=config_with_search,
    )
    rc = response.candidates[0]
    return show_response(rc)

#google_ground_search("how to do jump start in bmw x3?")

# Agent


In [14]:
# Car Assistant Agent with RAG and Gemini Grounded Search using LangGraph
# This notebook implements a car assistant that:
# 1. First tries to answer using a RAG system with local knowledge
# 2. Falls back to Gemini with Google Search grounding if RAG fails or returns "I cannot answer"

import os
import json
from typing import Dict, List, Optional, Any, TypedDict, Annotated, Literal
from dataclasses import dataclass
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader, DirectoryLoader
import google.generativeai as genai
from google.api_core import client_options as client_options_lib
from google.generativeai import types
from IPython.display import Markdown, display
import pandas as pd

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
#from langgraph.checkpoint import MemorySaver
from typing import TypeVar
from langgraph.pregel import Pregel
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

# Configuration for API keys
import dotenv
dotenv.load_dotenv()

# Set up Google API key
#GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
#if not GOOGLE_API_KEY:
#    raise ValueError("Please set GOOGLE_API_KEY in your environment variables or .env file")

# Set up Gemini configuration
genai.configure(api_key=GOOGLE_API_KEY)

# Define the agent state
class AgentState(TypedDict):
    """State for the car assistant agent workflow."""
    question: str
    rag_answer: Optional[str]
    search_answer: Optional[str]
    final_answer: Optional[str]
    source: Optional[str]
    should_use_search: bool
    status: Literal["RUNNING", "DONE"]

# Define the fallback detection function
def should_use_search(rag_answer: Optional[str]) -> bool:
    """Determine if we should fall back to search based on the RAG answer."""
    if not rag_answer:
        return True
    
    # List of exact phrases that indicate the RAG system couldn't provide a good answer
    fallback_exact_phrases = [
        "i cannot answer based on the provided information",
        "cannot answer based on the provided information",
        "i don't have enough information",
        "i do not have enough information",
        "insufficient information",
        "no information available",
        "not found in the provided context",
        "i don't know",
        "i do not know"
    ]
    
    rag_answer_lower = rag_answer.lower()
        
    # Check for exact phrases (substring match)
    for phrase in fallback_exact_phrases:
        if phrase in rag_answer_lower:
            return True
            
    # Additional checks for phrases indicating inability to answer
    if "cannot answer" in rag_answer_lower or "can't answer" in rag_answer_lower:
        return True
        
    if "sorry" in rag_answer_lower and ("don't" in rag_answer_lower or "cannot" in rag_answer_lower):
        return True
        
    if "unable to provide" in rag_answer_lower or "not able to provide" in rag_answer_lower:
        return True
            
    return False

# Define tools
class CarTools:
    """Tools for the car assistant agent."""
    
    @staticmethod
    def car_manual_rag(question: str) -> str:
        """Tool to query the RAG system with car manual knowledge."""
        return car_manual_rag(question)

    @staticmethod
    def google(query: str) -> str:
        """Tool to use Gemini with Google Search grounding to answer queries."""
        return google_ground_search(query)

    @staticmethod
    def image_search(image_url: str) -> str:
        """
        Tool to extract contents from an image based on the provided URL using the updated process_image function.
        """
        print(f"Processing image from URL: {image_url}")
        return process_image(image_url)  # Use the updated function



In [15]:

# Define node functions
def process_question(state: AgentState) -> AgentState:
    """Process the initial question and query the RAG system."""
    question = state["question"]
    rag_answer = CarTools.car_manual_rag(question)
    should_search = should_use_search(rag_answer)
    
    return {
        **state,
        "rag_answer": rag_answer,
        "should_use_search": should_search
    }

def route_by_answer_quality(state: AgentState) -> Literal["use_rag", "use_search"]:
    """Route based on whether we should use RAG results or search."""
    if state["should_use_search"]:
        return "use_search"
    else:
        return "use_rag"

def use_rag_answer(state: AgentState) -> AgentState:
    """Use the RAG answer as the final answer."""
    return {
        **state, 
        "final_answer": state["rag_answer"],
        "source": "Car Manual - RAG",
        "status": "DONE"
    }

def use_search_answer(state: AgentState) -> AgentState:
    """Use Google Search with Gemini to get an answer."""
    search_answer = CarTools.google(state["question"])
    return {
        **state,
        "search_answer": search_answer,
        "final_answer": search_answer,
        "source": "Gemini with Google Search Grounding",
        "status": "DONE"
    }

# Update the process_image_state function to return both image contents and search results
def process_image_state(state: AgentState) -> AgentState:
    """
    Processes an image URL, retrieves its description, and combines results
    for nearest gas stations and car service stations based on detected content.
    """
    image_url = state["question"]  # Assuming the question contains the image URL
    image_content = CarTools.image_search(image_url)  # Process the image URL

    # Initialize variables for results
    combined_result = f"Image Contents: {image_content}\n\n"

    # Check for "Low Fuel" in image contents
    if "Low Fuel" in image_content:
        gas_station_result = CarTools.google("Nearby gas stations in New York City")
        combined_result += f"Nearest Gas Stations: {gas_station_result}\n\n"

    # Check for "Warning Light" in image contents
    if "warning light" in image_content:
        car_service_result = CarTools.google("Nearby car service stations in New York City")
        combined_result += f"Nearest Car Service Stations: {car_service_result}\n\n"

    return {
        **state,
        "search_answer": combined_result,  # Combined result with both gas stations and service centers
        "final_answer": combined_result,  # Final response includes all data
        "source": "Image Search + Google Search",
        "status": "DONE"
    }
    

In [16]:

# Build the LangGraph
def build_car_assistant_graph():
    """Build the LangGraph for the car assistant."""
    # Initialize the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("process_question", process_question)
    workflow.add_node("process_image", process_image_state)  # Updated node
    workflow.add_node("use_rag_answer", use_rag_answer)
    workflow.add_node("use_search_answer", use_search_answer)
    workflow.add_node("route_by_answer_quality", route_by_answer_quality)
    
    # Add edges
    workflow.add_conditional_edges(
        "process_question",
        route_by_answer_quality,
        {
            "use_rag": "use_rag_answer",
            "use_search": "use_search_answer"
        }
    )
    
    # Add an edge for image processing
    workflow.add_edge("process_image", END)
    
    # Add edges to end the graph
    workflow.add_edge("use_rag_answer", END)
    workflow.add_edge("use_search_answer", END)
    
    # Set the entry point
    workflow.set_entry_point("process_question")
    
    return workflow.compile()

# Create the car assistant using LangGraph
car_assistant_graph = build_car_assistant_graph()

# Function Calling

In [17]:

# Function to run the car assistant and display results
def run_car_assistant(question: str, is_image: bool = False):
    """Run the car assistant for a given question or image URL and display the results."""
    # Initialize the state
    state = {
        "question": question,
        "rag_answer": None,
        "search_answer": None,
        "final_answer": None,
        "source": None,
        "should_use_search": False,
        "status": "RUNNING"
    }
    
    if is_image:
        # Use the process_image_state directly for image URLs
        result = process_image_state(state)
    else:
        # Run the graph for text-based questions
        result = car_assistant_graph.invoke(state)
    
    # Display the result
    display(Markdown(f"## Answer\n{result['final_answer']}"))
    print(f"Source: {result['source']}")
    return result

def smart_auto_assist_image(image_url):
    print("\n=== Car Assistant with Image ===\n")
    
    # Example questions and image URLs to test
    example_inputs = [(image_url, True)]
    
    # Testing the assistant with example inputs
    for i, (input_data, is_image) in enumerate(example_inputs, 1):
        print(f"\n{'='*80}\nInput {i}: {input_data}\n{'='*80}")
        result = run_car_assistant(input_data, is_image=is_image)


def smart_auto_assist_chat(query):
    print("\n=== Car Assistant with Chat ===\n")
    
    # Example questions and image URLs to test
    example_inputs = [
        (query, False)
    ]
    
    # Testing the assistant with example inputs
    for i, (input_data, is_image) in enumerate(example_inputs, 1):
        print(f"\n{'='*80}\nInput {i}: {input_data}\n{'='*80}")
        result = run_car_assistant(input_data, is_image=is_image)
        

# Interactive mode for custom questions
def interactive_mode():
    while True:
        question = input("\nAsk a car-related question (or type 'exit' to quit): ")
        if question.lower() == 'exit':
            break
        run_car_assistant(question)

# Uncomment the next line to enter interactive mode
# interactive_mode()

# Testing

## **UseCase: 1 - Chat with car manual (RAG)** 

In [18]:
smart_auto_assist_chat("where to find VIN?")


=== Car Assistant with Chat ===


Input 1: where to find VIN?

🤔 Query: where to find VIN?

✅ Answer:
The VIN is located in the engine compartment, on the right-hand side of the vehicle.


## Answer
The VIN is located in the engine compartment, on the right-hand side of the vehicle.

Source: Car Manual - RAG


## **UseCase: 2 - Chat with Google - If RAG has no Answer**

In [19]:
smart_auto_assist_chat("what is the difference between AWD and FWD?")


=== Car Assistant with Chat ===


Input 1: what is the difference between AWD and FWD?

🤔 Query: what is the difference between AWD and FWD?

✅ Answer:
I cannot answer based on the provided information.  The text mentions all-wheel drive (AWD) and its benefit of improved drive power, but it does not explain the difference between AWD and front-wheel drive (FWD).


## Answer
The key difference between All-Wheel Drive (AWD) and Front-Wheel Drive (FWD) lies in which wheels receive power from the engine:

*   **FWD (Front-Wheel Drive):** The engine sends power only to the front wheels.
*   **AWD (All-Wheel Drive):** The engine sends power to all four wheels.

Here's a more detailed breakdown:

**FWD (Front-Wheel Drive)**

*   **How it works:** The engine's power is directed to the front wheels, which are responsible for both propelling the vehicle and steering.
*   **Benefits:**
    *   Generally more fuel-efficient due to lighter weight and fewer components.
    *   Lower production costs, often resulting in a lower purchase price.
    *   Predictable handling in most normal driving conditions.
*   **Cons:**
    *   Less traction than AWD, especially in slippery conditions.
    *   The front wheels can experience "torque steer" (a pulling sensation) during hard acceleration.

**AWD (All-Wheel Drive)**

*   **How it works:** The engine sends power to all four wheels. There are different types of AWD systems:
    *   **Full-time AWD:** Power is constantly supplied to all four wheels.
    *   **Part-time AWD:** The system primarily operates in two-wheel drive (usually FWD) and automatically engages all four wheels when it detects a loss of traction.
*   **Benefits:**
    *   Enhanced traction, providing better grip on slippery surfaces like snow, ice, and wet roads.
    *   Improved acceleration and stability, especially when cornering.
    *   Superior performance in light off-road situations.
*   **Cons:**
    *   Generally less fuel-efficient than FWD due to added weight and complexity.
    *   Higher purchase price and potentially higher maintenance costs.
    *   Can sometimes reduce legroom due to the placement of transmission components.

**Which is better?**

*   **FWD:** Best for drivers who prioritize fuel efficiency, affordability, and live in areas with mild weather conditions.
*   **AWD:** Best for drivers who need maximum traction and stability, especially those who live in areas with harsh weather (snow, ice, rain) or who occasionally drive on unpaved roads.


Source: Gemini with Google Search Grounding


# **Testing - Image Analysis - Upload dashboard image**

**Sample Image:** 

![](https://raw.githubusercontent.com/argowthamkumar/kaggle/refs/heads/main/capstone/images/low_fuel.jpg)

In [20]:
smart_auto_assist_image("https://raw.githubusercontent.com/argowthamkumar/kaggle/refs/heads/main/capstone/images/low_fuel.jpg")


=== Car Assistant with Image ===


Input 1: https://raw.githubusercontent.com/argowthamkumar/kaggle/refs/heads/main/capstone/images/low_fuel.jpg
Processing image from URL: https://raw.githubusercontent.com/argowthamkumar/kaggle/refs/heads/main/capstone/images/low_fuel.jpg


## Answer
Image Contents: Here's a description of the image contents:

The image shows the dashboard of a car. There are two main gauges:

*   **Left Gauge:** Displays the engine's RPM (revolutions per minute) in thousands. The needle indicates a little over 1000 RPM. There is also a temperature gauge, which reads as slightly cold.
*   **Right Gauge:** Displays the vehicle's speed in kilometers per hour (km/h). The needle is at approximately 20 km/h. Additionally, a fuel gauge indicates the fuel level is very low.

In the center of the dashboard, between the gauges, there is a digital display showing:

*   A "Low Fuel" warning message with a fuel pump icon
*   The transmission is in "Drive" (D)
*   The outside temperature is -1 degrees Celsius.
*   The car has traveled 1041 km.

There are several warning lights illuminated on the dashboard:

*   Oil Pressure Warning Light: **HIGH**
*   Battery Warning Light: **HIGH**
*   Seatbelt Warning Light: **MEDIUM**
*   Check Engine Light: **MEDIUM**
*   Tire Pressure Warning Light: **MEDIUM**

Nearest Gas Stations: Okay, I can help you find nearby gas stations in New York City. To give you the most relevant results, I need to know your current location within New York City.

However, I can provide you with some general information and options:

**General Options for Finding Gas Stations:**

*   **Use Online Search Engines/Apps:** Use online search engines such as Google, DuckDuckGo, or Maps. Many of these have the functionality to search for "gas stations near me".
*   **Use Gas Station Finder Apps/Websites:** There are specific apps and websites designed to locate gas stations and compare prices, such as GasBuddy, and Way.
*   **Check Specific Gas Station Websites:** You can use the Exxon Mobil Fuel Finder to find Exxon and Mobil stations. Also, you can find Sunoco stations.

**Some Gas Stations Listed in New York, NY:**

*   **Shell:** Brooklyn (98 3rd Ave)
*   **BP:** Brooklyn (677 Kent Ave)
*   **Mobil:** Manhattan (51-63 8th Ave)
*   **Sunoco:** 5080 Broadway, New York, NY 10034

To get the most accurate results, I recommend using one of the methods above and providing your specific location in New York City.


Nearest Car Service Stations: Okay, I can help you find car service stations in New York City. Here are a few options, based on the search results:

**AAA Approved Auto Repair Facilities** (Note: You may need a AAA membership)

*   **Pica's Automotive Service:** 90 NJ-139, Jersey City, NJ 07310
*   **Audi Brooklyn:** 700 Hicks St, Brooklyn, NY 11231
*   **Centro Auto Body and Repair:** 334 Hoboken Ave, Jersey City, NJ 07306
*   **Urban Classics Auto Repair:** 56 Kosciuszko St, Brooklyn, NY 11205
*   **West Side Tire & Auto:** 236 West Side Ave, Jersey City, NJ 07305

**Other Options Listed**

*   **2020 Auto Service Corp:** 2315 12th Ave, New York, NY 10027
*   **Manhattan Auto Repair Inc:** 552 W 48th St, New York, NY 10036
*   **Manhattan Alignment & Diagnostic Center:** 555 W 131st Street, New York, NY 10027 (specializing in foreign and domestic automobiles as well as hybrids)
*   **Luxury Auto Service:** 711 11th Ave, New York, NY 10019 (specializing in exotic car repairs)

**Goodyear Service Centers / Warren Tire Service Centers** (Primarily Upstate NY)

Keep in mind that "nearby" is relative to your current location within New York City. To get the most accurate results, I recommend performing a search on Google Maps or a similar service, as I don't have access to your precise location.




Source: Image Search + Google Search
